In [14]:
import arviz as az
import matplotlib.pyplot as plt
import pyjags as pj
import pandas as pd
import scipy.stats as ss
import numpy as np
from IPython.display import Image

### **1. Bayesian hierarchical normal linear regression - Univariate**

#### 1a. Univariate prior formulation

In [15]:
df = pd.read_csv('../data/lifeexpdiff.csv', sep=',', header=0, index_col=[0])
years = np.array([1960, 1970, 1980, 1990, 2000, 2010])
data = {'Y': np.ma.masked_invalid(df.values),
        'X': np.ma.masked_invalid(years),
        'Xbar': np.ma.masked_invalid(np.mean(years))}

inits = [{'tausq.y': 1, 'beta1': 0, 'beta2': 0, 'sigma.alpha1': 1, 'sigma.alpha2': 1},
             {'tausq.y': 100, 'beta1': 100, 'beta2': 100, 'sigma.alpha1': 0.1, 'sigma.alpha2': 0.1},
             {'tausq.y': 0.01, 'beta1': -100, 'beta2': -100, 'sigma.alpha1': 10, 'sigma.alpha2': 10}]


In [16]:
jags_model_string = '''
  data {
      dim.Y <- dim(Y)
    }

    model {
      for(i in 1:dim.Y[1]) {

        for(j in 1:dim.Y[2]) {
          Y[i,j] ~ dnorm(mu[i,j], tausq.y)
          mu[i,j] <- alpha[i,1] + alpha[i,2] * (X[j] - Xbar)
        }

        alpha[i,1] ~ dnorm(beta1, 1 / sigma.alpha1^2)
        alpha[i,2] ~ dnorm(beta2, 1 / sigma.alpha2^2)
      }

      tausq.y ~ dgamma(0.001, 0.001)
      sigma.y <- 1 / sqrt(tausq.y)

      beta1 ~ dnorm(0.0, 1.0E-6)
      beta2 ~ dnorm(0.0, 1.0E-6)
      sigma.alpha1 ~ dexp(0.001)
      sigma.alpha2 ~ dexp(0.001)
    }
    '''

In [17]:
jags_model \
    = pj.Model(code=jags_model_string,
               init=inits,
               data=data,
               chains=3,
               )

adapting: iterations 3000 of 3000, elapsed 0:00:00, remaining 0:00:00


#### 1b. Univariate model summary

In [18]:
# Usin 100000 iterations
samples_1 = jags_model.sample(iterations=140000, vars=["beta1", "beta2", "sigma.y", "sigma.alpha1", "sigma.alpha2"])
idata = az.from_pyjags(samples_1)

sampling: iterations 117498 of 420000, elapsed 0:00:06, remaining 0:00:15
sampling: iterations 323685 of 420000, elapsed 0:00:16, remaining 0:00:05
sampling: iterations 420000 of 420000, elapsed 0:00:20, remaining 0:00:00


In [19]:
func_dict = {
    "Mean":np.mean,
    "SD": np.std,
    "SE": lambda x: np.std(x) / np.sqrt(np.size(x)),
    "2.5%": lambda x: np.percentile(x, 2.5),
    "25%": lambda x: np.percentile(x, 25),
    "50%": lambda x: np.percentile(x, 50),
    "75%": lambda x: np.percentile(x, 75),
    "97.5%": lambda x: np.percentile(x, 97.5),
}

In [20]:
# burning the first 40000 iterations
az.summary(idata.posterior.sel(draw=slice(40000, 140000))[["beta1", "beta2", "sigma.y", "sigma.alpha1", "sigma.alpha2"]], stat_funcs=func_dict)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,Mean,SD,SE,2.5%,25%,50%,75%,97.5%
beta1,0.117,0.191,-0.239,0.480,0.0,0.0,292438.0,290572.0,1.0,0.117,0.191,0.0,-0.258,-0.010,0.117,0.243,0.493
beta2,-0.005,0.004,-0.012,0.002,0.0,0.0,233401.0,271002.0,1.0,-0.005,0.004,0.0,-0.013,-0.008,-0.005,-0.003,0.002
sigma.y,0.382,0.019,0.347,0.419,0.0,0.0,148465.0,216417.0,1.0,0.382,0.019,0.0,0.347,0.369,0.382,0.395,0.423
sigma.alpha1,1.330,0.141,1.077,1.597,0.0,0.0,156939.0,139746.0,1.0,1.330,0.141,0.0,1.088,1.231,1.318,1.416,1.639
sigma.alpha2,0.024,0.003,0.019,0.030,0.0,0.0,111824.0,134482.0,1.0,0.024,0.003,0.0,0.019,0.022,0.024,0.026,0.031


#### 1c.The 95% equal-tailed posterior credible intervals for $\beta_1$ and $\beta_2$

* 95% posterior predictive interval for $\beta_1$: [-0.258, 0.491]
* 95% posterior predictive interval for $\beta_2$: [-0.013, 0.002]

The 95% equal-tailed posterior credible intervals for $\beta_1$ and $\beta_2$

### **2. Bayesian hierarchical normal linear regression - Bivariate**

#### 2a. Bivariate prior formulation

In [21]:
data = {'Y': np.ma.masked_invalid(df.values),
        'X': np.ma.masked_invalid(years),
        'Xbar': np.ma.masked_invalid(np.mean(years)),
        'Omega0': np.ma.masked_invalid(np.array([[1, 0], [0, 0.0005]])),
        'mu0': np.ma.masked_invalid(np.array([0, 0])),
        'Sigma0.inv': np.ma.masked_invalid(np.array([[1e-6, 0], [0, 1e-6]]))}

inits = [{'tausq.y': 1, 'beta': np.array([0, 0]), 'Omega.inv': np.identity(2)},
         {'tausq.y': 100, 'beta': np.array([100, 100]), 'Omega.inv': 100 * np.identity(2)},
         {'tausq.y': 0.1, 'beta': np.array([-100, -100]), 'Omega.inv': 0.01 * np.identity(2)}]

In [22]:
jags_model_string = '''
  data {
      dim.Y <- dim(Y)
    }
    model {
      for(i in 1:dim.Y[1]) {

        for(j in 1:dim.Y[2]) {
          Y[i,j] ~ dnorm(mu[i,j], tausq.y)
          mu[i,j] <- alpha[i,1] + alpha[i,2] * (X[j] - Xbar)
        }

        alpha[i,1:2] ~ dmnorm(beta, Omega.inv)
      }

      tausq.y ~ dgamma(0.001, 0.001)
      sigma.y <- 1 / sqrt(tausq.y)

      beta ~ dmnorm(mu0, Sigma0.inv)
      Omega.inv ~ dwish(2*Omega0, 2)
      Omega <- inverse(Omega.inv)

      rho <- Omega[1,2] / sqrt(Omega[1,1] * Omega[2,2])
    }
    '''


In [23]:
jags_model \
    = pj.Model(code=jags_model_string,
               init=inits,
               data=data,
               chains=3,
               )

In [24]:
samples_1 = jags_model.sample(iterations=14000, vars=["beta", "sigma.y", "rho", 'Omega', 'alpha'])
samples_1['Omega'] = np.reshape(samples_1['Omega'], [4, 14000, 3])
samples_1['alpha'] =  np.reshape(samples_1['alpha'], [100, 14000, 3])

sampling: iterations 42000 of 42000, elapsed 0:00:03, remaining 0:00:00


In [25]:
idata = az.from_pyjags(samples_1)

#### 2b. Bivariate model summary

In [26]:
# burning the first 4000 iterations
az.summary(idata.posterior.sel(draw=slice(4000, 14000))[["beta", "sigma.y", "rho", 'Omega', 'alpha']], stat_funcs=func_dict)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,Mean,SD,SE,2.5%,25%,50%,75%,97.5%
beta[0],0.116,0.189,-0.236,0.472,0.001,0.001,28984.0,28951.0,1.0,0.116,0.189,0.001,-0.254,-0.010,0.117,0.243,0.485
beta[1],-0.005,0.004,-0.012,0.002,0.000,0.000,23650.0,26902.0,1.0,-0.005,0.004,0.000,-0.013,-0.008,-0.005,-0.003,0.002
sigma.y,0.383,0.019,0.346,0.418,0.000,0.000,15157.0,21717.0,1.0,0.383,0.019,0.000,0.347,0.370,0.382,0.395,0.423
rho,0.175,0.147,-0.103,0.444,0.001,0.001,21653.0,25475.0,1.0,0.175,0.147,0.001,-0.118,0.076,0.179,0.278,0.451
Omega[0],1.753,0.370,1.124,2.456,0.002,0.002,27678.0,28776.0,1.0,1.753,0.370,0.002,1.174,1.491,1.705,1.958,2.612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
alpha[95],-0.010,0.009,-0.027,0.006,0.000,0.000,29415.0,29438.0,1.0,-0.010,0.009,0.000,-0.027,-0.016,-0.010,-0.005,0.007
alpha[96],-1.498,0.155,-1.786,-1.202,0.001,0.001,29537.0,29166.0,1.0,-1.498,0.155,0.001,-1.802,-1.602,-1.499,-1.393,-1.192
alpha[97],-0.042,0.009,-0.058,-0.026,0.000,0.000,27855.0,28233.0,1.0,-0.042,0.009,0.000,-0.058,-0.048,-0.042,-0.036,-0.025
alpha[98],0.020,0.155,-0.273,0.310,0.001,0.001,30078.0,29688.0,1.0,0.020,0.155,0.001,-0.284,-0.083,0.019,0.123,0.324


#### 2c. The posterior probability for $\rho > 0$